In [2]:
import os
import face_recognition
import numpy as np
import pickle
import os
import face_recognition
import cv2
import numpy as np
import pandas as pd
from datetime import datetime, time
from tensorflow.keras.models import model_from_json


In [ ]:


with open('emotion_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

emotion_model = model_from_json(loaded_model_json)

emotion_model.load_weights('emotion_model_weights.h5')

emotion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Emotion detection model loaded successfully.")


Emotion detection model loaded successfully.


Optimizer: 'adam' - adaptive learning rate algorithm, good for most deep learning tasks.

Loss function: 'categorical_crossentropy' - standard for multi-class classification (emotions are likely categories like happy, sad, angry, etc.).

Metrics: ['accuracy'] - tracks classification accuracy during evaluation.

In [ ]:

dataset_path = "Student/Original Images"

known_face_encodings = []
known_face_names = []

for student_name in os.listdir(dataset_path):
    student_folder = os.path.join(dataset_path, student_name)
    if not os.path.isdir(student_folder):
        continue

    for image_name in os.listdir(student_folder):
        image_path = os.path.join(student_folder, image_name)

        image = face_recognition.load_image_file(image_path) #Loads image from disk into numpy array format
        face_locations = face_recognition.face_locations(image) #Tells the system where faces are before encoding them
        face_encodings = face_recognition.face_encodings(image, face_locations)  #Converts detected faces into 128-dimensional vectors (numerical fingerprints)
 
        for encoding in face_encodings:
            known_face_encodings.append(encoding)
            known_face_names.append(student_name)

with open("face_encodings.pkl", "wb") as f:
    pickle.dump((known_face_encodings, known_face_names), f) #Converts Python objects to binary format for storage.

print("Face recognition dataset prepared and saved successfully.")


Face recognition dataset prepared and saved successfully.


In [ ]:


with open("face_encodings.pkl", "rb") as f:
    known_face_encodings, known_face_names = pickle.load(f)

with open('emotion_model.json', 'r') as json_file:
    loaded_model_json = json_file.read()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights('emotion_model_weights.h5')
emotion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

emotion_labels = ['surprise', 'fear', 'disgust', 'happy', 'sad', 'angry', 'neutral']

## Testing

In [ ]:
attendance = {}

start_time = time(9, 30)
end_time = time(10, 0)

def is_within_time_window():
    current_time = datetime.now().time()
    return start_time <= current_time <= end_time

def detect_and_mark_attendance(test_images_folder="Student/atest", output_csv="attendance.csv"):
    for image_name in os.listdir(test_images_folder):
        image_path = os.path.join(test_images_folder, image_name)
        image = face_recognition.load_image_file(image_path)
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

            name = "Unknown"
            if matches:
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

            if name != "Unknown" and is_within_time_window():
                face_img = image_bgr[top:bottom, left:right]
                face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)  
                face_img = cv2.resize(face_img, (48, 48))  
            
                face_img = face_img.astype('float32') / 255.0
                face_img = np.expand_dims(face_img, axis=0)
                face_img = np.expand_dims(face_img, axis=-1)

                emotion_prediction = emotion_model.predict(face_img)
                emotion_label = emotion_labels[np.argmax(emotion_prediction)]

                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                attendance[name] = {"Time": timestamp, "Status": "Present", "Emotion": emotion_label}

    # After processing images, mark absent students
    all_students = set(known_face_names)
    present_students = set(attendance.keys())
    for student in all_students - present_students:
        attendance[student] = {"Time": "", "Status": "Absent", "Emotion": ""}


    attendance_df = pd.DataFrame.from_dict(attendance, orient='index')
    attendance_df.index.name = 'Student Name'
    attendance_df.to_csv(output_csv)

    print(f"Attendance marked and saved to {output_csv}")

detect_and_mark_attendance()

1/1 [==============================] - 0s 21ms/step
Attendance marked and saved to attendance.csv
